<a href="https://colab.research.google.com/github/jacr2006/android-tflitemodel-drowsiness-detection/blob/master/data_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorboardcolab

In [3]:
import numpy as np
from tensorboardcolab import TensorBoardColab

Using TensorFlow backend.


Normalizacion de los datos

In [6]:
N_LANDMARK = 68# numero de marcas faciales
INPUT_SHAPE = (96, 96, 1)# ancho, alto y canales de la imagen
OUTPUT_SIZE = N_LANDMARK*2# tamaño de la capa de salida, dos coordenadas por cada marca facial

# lectura de los dataser previamente procesados
PATH_DRIVE = '/content/drive/My Drive/Proyectos Nuevos/ML/multiclass_landmark_keras_cnn/'
X = np.load(PATH_DRIVE + "img_dataset.npy")
y = np.load(PATH_DRIVE + "pts_dataset.npy")
print(y.shape)
y = y.reshape(-1, OUTPUT_SIZE)

# escalada de la data de entrada
X = X/255.0#normalizacion de los datos de las imagenes entre (0,1)

_mean = np.mean(y)
_max = np.amax(y)
_min = np.amin(y)
print(str(_mean))
print(str(_max))
print(str(_min))
y = (y-_mean)/(_max-_min)# normalizacion de los datos las marcas faciales entre (-1,1)

print(X.shape)
print(y.shape)


(6666, 68, 2)
48.55575925239583
122.0
-24.0
(6666, 96, 96, 1)
(6666, 136)


Visualizacion de la data normalizada

In [7]:
print(X[100,:,:,0])

[[0.18039216 0.15294118 0.14117647 ... 0.14509804 0.10196078 0.12941176]
 [0.17254902 0.16470588 0.14509804 ... 0.15294118 0.09019608 0.25882353]
 [0.18431373 0.17254902 0.16862745 ... 0.23137255 0.15686275 0.11764706]
 ...
 [0.16470588 0.16470588 0.17254902 ... 0.12941176 0.09411765 0.16862745]
 [0.16078431 0.16470588 0.18823529 ... 0.13333333 0.10196078 0.17647059]
 [0.14509804 0.17647059 0.16470588 ... 0.13333333 0.11764706 0.12941176]]


In [8]:
print(y[100,:])

[-0.30517643 -0.12024493 -0.30517643 -0.03120383 -0.29832712  0.05783727
 -0.28462849  0.14002905 -0.25038191  0.21537151 -0.19558739  0.27701535
 -0.12709424  0.3455085  -0.05175178  0.38660439  0.03044001  0.40030302
  0.10578247  0.37975507  0.15372768  0.32496055  0.2085222   0.26331672
  0.2564674   0.19482357  0.29071398  0.1194811   0.30441261  0.05783727
  0.31811124 -0.02435452  0.32496055 -0.09284767 -0.22983397 -0.17503945
 -0.18188876 -0.19558739 -0.12709424 -0.20928602 -0.06545041 -0.20928602
 -0.01065589 -0.18188876  0.10578247 -0.17503945  0.15372768 -0.20928602
  0.21537151 -0.20243671  0.26331672 -0.18873808  0.29756329 -0.14764219
  0.05098795 -0.1065463   0.05098795 -0.03805315  0.05098795  0.02359069
  0.05098795  0.09208384 -0.02435452  0.11263179  0.00989206  0.1194811
  0.04413864  0.13317973  0.07838521  0.1194811   0.10578247  0.10578247
 -0.17503945 -0.1065463  -0.13394356 -0.12709424 -0.09284767 -0.12709424
 -0.05175178 -0.09284767 -0.09284767 -0.08599835 -0.

Estructura de la red neuronal (CNN)

In [9]:
import keras
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, AveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation, Dense
from keras.layers import Flatten, Input, Convolution2D, add, Dropout, MaxPool2D
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=INPUT_SHAPE))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(OUTPUT_SIZE))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.optimizers import Adam
import numpy as np
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

NUM_EPOCHS = 100# numero de epocas de entrenamiento
INIT_LR = 0.001# coeficiente de aprendizaje inicial
BS = 256# se inicio el entrenamiento con un tamaño de 32 y se aumento hasta lograr la convergencia

# entrenar una CNN para detectar marcas faciales es distinto de predecir clases (binaria o multiclase)
# en vez de obtener una distribucion de clases usando una funcion de perdida como "entropy loss" que mide la capacidad de clasificacion,
# se debe usar una fucnion de perdida para regresion, la cual compara directamente el valor inferido con el real 

#opt = SGD(lr=INIT_LR, momentum=0.9)# Gradiente descendete estoscastico como metodo de optimizacion
opt = Adam(lr=INIT_LR, decay=1e-3 / 200)

# compilacion del modelo
model.compile(loss='mean_squared_error', optimizer=opt, metrics=["mae"])

tbc=TensorBoardColab()
callbacks_list = [TensorBoardColabCallback(tbc)]



Wait for 8 seconds...
TensorBoard link:
https://fa4c4e2b.ngrok.io


In [11]:
H = model.fit(
    X, y, batch_size=BS, 
    epochs=NUM_EPOCHS, shuffle=True,
    verbose=1, 
    validation_split = 0.2,
    callbacks=callbacks_list)

model.save(PATH_DRIVE+'models/model2.h5')




Train on 5332 samples, validate on 1334 samples


Epoch 1/100
5332/5332 [==============================] - 13s 2ms/step - loss: 0.2435 - mean_absolute_error: 0.2629 - val_loss: 0.0435 - val_mean_absolute_error: 0.1760

Epoch 2/100
5332/5332 [==============================] - 4s 717us/step - loss: 0.0259 - mean_absolute_error: 0.1343 - val_loss: 0.0409 - val_mean_absolute_error: 0.1685
Epoch 3/100
5332/5332 [==============================] - 4s 719us/step - loss: 0.0227 - mean_absolute_error: 0.1248 - val_loss: 0.0385 - val_mean_absolute_error: 0.1607
Epoch 4/100
5332/5332 [==============================] - 4s 718us/step - loss: 0.0197 - mean_absolute_error: 0.1150 - val_loss: 0.0364 - val_mean_absolute_error: 0.1529
Epoch 5/100
5332/5332 [==============================] - 4s 719us/step - loss: 0.0169 - mean_absolute_error: 0.1054 - val_loss: 0.0346 - val_mean_absolute_error: 0.1455
Epoch 6/100
5332/5332 [==============================] - 4s 722us/step - loss: 0.0145 - mean_absolute_e